In [48]:
# face verification with the VGGFace2 model
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import pandas as pd

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    print(filename)
    pixels = pyplot.imread(filename)
    pyplot.imshow(pixels)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    print(results)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

model = None
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create a vggface model
    global model
    if not model:
        model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat

# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
        return True, score
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
        return False, score

In [49]:
import dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data\\shape_predictor_68_face_landmarks.dat")

def extract_face2(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_locations = detector(gray, 1)
    return face_locations

In [61]:
import cv2

In [50]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd

In [51]:
BASE_PATH = pathlib.Path('../data/mfr2')
labels_path = BASE_PATH / 'mfr2_labels.txt'
pairs_path = BASE_PATH / 'pairs.txt'

In [52]:
labels=pd.read_csv(filepath_or_buffer=str(labels_path))
labels=labels.apply(lambda x: x.apply(lambda y: y.strip() if type(y) == type('') else y), axis=0)
people=labels["person"].unique()

In [53]:
labels[(labels["person"]==people[0]) & (labels["mask"]=="no-mask")]

,person,index,mask
1,AdrianDunbar,2,no-mask
3,AdrianDunbar,4,no-mask


In [54]:
list(labels[(labels["person"]==people[0]) & (labels["mask"]=="no-mask")]["index"])

[2, 4]

In [55]:
def get_indexes(person, mask=True):
    if mask:
        mask_filter = labels["mask"]!="no-mask"
    else:
        mask_filter = labels["mask"]=="no-mask"
    person_filter = labels["person"]==person
    return list(labels[person_filter & mask_filter]["index"])

In [56]:
get_indexes(people[0],mask=False)

[2, 4]

In [57]:
pairs={person:list(zip(get_indexes(person,mask=True), get_indexes(person, mask=False))) for person in people}

In [58]:
sum([len(p) for p in pairs.values()])

91

In [59]:
person, pairs = list(pairs.items())[0]
index1, index2 = pairs[0]
path1 = BASE_PATH / person / f'{person}_{str(index1).rjust(4, "0")}.png'
path2 = BASE_PATH / person / f'{person}_{str(index2).rjust(4, "0")}.png'

In [65]:
#extract_face2(str(path1))
extract_face2(str(path2))

rectangles[[(26, 46) (116, 136)]]

In [44]:
results = []

# positive test
for person, pairs in pairs.items():
    for index1, index2 in pairs:
        path1 = BASE_PATH / person / f'{person}_{str(index1).rjust(4, "0")}.png'
        path2 = BASE_PATH / person / f'{person}_{str(index2).rjust(4, "0")}.png'

        features1,features2=get_embeddings([path1, path2])
        matched, distance = is_match(features1, features2)

        results.append([pair, matched, distance])

NameError: name 'person1' is not defined

In [ ]:
TP=len([x for x in results if x[1]]) / len(results)
FN=1-TP

In [ ]:
TP

In [ ]:
FN

In [ ]:
len(list(BASE_PATH.iterdir()))

In [ ]:
results_neg = []
count = 0

for person1 in BASE_PATH.iterdir():
    for person2 in BASE_PATH.iterdir():
        if not person1.is_dir() or not person2.is_dir() or person1.name == person2.name or not person1.name.isalpha() or not person2.name.isalpha():
            continue
        count += 1
        if count >= len(results):
            break
        index1 = index2 = 1
        
        path1 = BASE_PATH / person1.name / f'{person1.name}_{str(index1).rjust(4, "0")}.png'
        path2 = BASE_PATH / person2.name / f'{person2.name}_{str(index2).rjust(4, "0")}.png'

        try:
            features1,features2=get_embeddings([path1, path2])
            matched, distance = is_match(features1, features2)
        except Exception as e:
            print(e)
            continue

        results_neg.append([pair, matched, distance])

In [ ]:
FP=len([x for x in results_neg if x[1]]) / len(results_neg)
TN=1-FP

In [ ]:
FP

In [ ]:
TN

In [ ]:
TP=len([x for x in results if x[1]]) / (len(results) + len(results_neg))
FN=len([x for x in results if not x[1]]) / (len(results) + len(results_neg))
FP=len([x for x in results_neg if x[1]]) / (len(results) + len(results_neg))
TN=len([x for x in results_neg if not x[1]]) / (len(results) + len(results_neg))
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

array = [[TP,FN],
         [FP,TN]]

df_cm = pd.DataFrame(array, ['same', 'different'], ['verified', 'unverified'])
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()

In [ ]:
from sklearn import svm
X = [x[2] for x in results] + [x[2] for x in results_neg]
y = [1 for x in results] + [0 for x in results_neg]
clf = svm.SVC()
clf.fit(np.array(X).reshape(-1, 1), y)

In [ ]:
ps=clf.predict(np.array([x[2] for x in results]).reshape(-1, 1))

In [ ]:
len([x for x in ps if x == 1])

In [ ]:
ns=clf.predict(np.array([x[2] for x in results_neg]).reshape(-1, 1))

In [ ]:
np.array([x[2] for x in results + results_neg])

In [ ]:
len([x for x in ps if x == 0])

In [ ]:
len(ps)

In [ ]:
TP=len([x for x in ps if x]) / (len(results) + len(results_neg))
FN=len([x for x in ps if not x]) / (len(results) + len(results_neg))
FP=len([x for x in ns if x]) / (len(results) + len(results_neg))
TN=len([x for x in ns if not x]) / (len(results) + len(results_neg))
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

array = [[TP,FN],
         [FP,TN]]

df_cm = pd.DataFrame(array, ['same', 'different'], ['verified', 'unverified'])
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()

In [ ]:
TP

In [ ]:
FN

In [ ]:
FP

In [ ]:
len(results) + len(results_neg)

In [ ]:
get_embeddings([r"D:\self\university\workshop\project\data\mfr2\AdrianDunbar\AdrianDunbar_0002.png"])

In [ ]:
get_embeddings([r"D:\self\university\workshop\project\MaskedFaceDetection\data\Nicolas_Cage_unmasked1.jpg"])